In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage

llm = ChatOpenAI(model='gpt-4o-mini')

llm.invoke([HumanMessage("잘지냈어?")])

AIMessage(content='네, 잘 지냈어요! 당신은 어떻게 지내셨어요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 12, 'total_tokens': 28, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DAXcHrsdIxlgeMEEYep2ObdeZ8t6h', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6ff5-4fa9-7910-880f-3d0736aeeaed-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens': 16, 'total_tokens': 28, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [6]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # @tool 데코레이터를 사용하여 함수를 도구로 등록 
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수
    
    Args:
        timezone (str): 타임존(예: Asia/Seoul). 실제 존재해야 함 
        location (str): 지역명. 타임존은 모든 지명에 대응하지 않으므로 이후 llm 답변 생성에 사용됨
    """

    tz = pytz.timezone(timezone)

    now = datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')

    location_and_local_time = f'{timezone} ({location}) 현재 시각 {now}' # 타임존, 지역명, 현재 시각을 문자열로 반환

    print(location_and_local_time)

    return location_and_local_time

In [7]:
# 도구를 tools 리스트에 추가하고, tool_dict에도 추가 
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# 도구를 모델에 바인딩 : 모델에 도구를 바인딩하면, 도구를 사용하여 답변을 생성할 수 있음 
llm_with_tools = llm.bind_tools(tools)

In [8]:
from langchain_core.messages import SystemMessage

messages = [
    SystemMessage("너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage("부산은 지금 몇시야?")
]

response = llm_with_tools.invoke(messages)
messages.append(response)

print(messages)

[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 129, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAXig1yCnJ4Zt7rVSVtKikTzRGDdH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6ffb-5b5f-7813-ad0a-24ebb6521279-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_wfVs6CJIZyiwwuvjowfWb8b6', 'type': 'tool_call'}],

In [9]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call['name']]
    print(tool_call['args'])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2026-02-18 18:01:15


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 129, 'total_tokens': 152, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAXig1yCnJ4Zt7rVSVtKikTzRGDdH', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c6ffb-5b5f-7813-ad0a-24ebb6521279-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_wfVs6CJIZyiwwuvjowfWb8b6', 'type': 'tool_call'}

In [10]:
llm_with_tools.invoke(messages)

AIMessage(content='부산은 현재 2026년 2월 18일 18시 01분 15초입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 185, 'total_tokens': 211, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_324beb61c9', 'id': 'chatcmpl-DAXipkcNscWB5WKpq43NIyeabnsVa', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6ffb-7cda-72b2-adb9-e3d6c88fdebf-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 185, 'output_tokens': 26, 'total_tokens': 211, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [11]:
from pydantic import BaseModel, Field

class StockHistoryInput(BaseModel):
    ticker: str = Field(..., title="주식 코드", description="주식 코드 (예: AAPL)")
    period: str = Field(..., title="기간", description="주식 데이터 조회 기간 (예: id, 1mo, 1y)")

In [12]:
import yfinance as yf

@tool
def get_yf_stock_history(stock_history_input: StockHistoryInput) -> str:
    """주식 종목의 가격 데이터를 조회하는 함수"""
    stock = yf.Ticker(stock_history_input.ticker)
    history = stock.history(period=stock_history_input.period)
    history_md = history.to_markdown()

    return history_md

tools = [get_current_time, get_yf_stock_history]
tool_dict = {
    "get_current_time": get_current_time,
    "get_yf_stock_history": get_yf_stock_history
}

llm_with_tools = llm.bind_tools(tools)

In [13]:
messages.append(HumanMessage("테슬라는 한 달 전에 비해 주가각 올랐나 내렸나?"))

response = llm_with_tools.invoke(messages)

print(response)

messages.append(response)

content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 274, 'total_tokens': 301, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DAXj0BIYXUodmAUpD82JMasMWZNVM', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--019c6ffb-ab34-7962-a91a-4e37b37c056d-0' tool_calls=[{'name': 'get_yf_stock_history', 'args': {'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}, 'id': 'call_mpt7sqHzKDh6wN9qftUK9Yp9', 'type': 'tool_call'}] invalid_tool_calls=[] usage_metadata={'input_tokens': 274, 'output_tokens': 27, 'total_tokens': 301, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audi

In [14]:
for tool_call in response.tool_calls:
    selected_tool = tool_dict[tool_call['name']]
    print(tool_call['args'])
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)
    print(tool_msg)

{'stock_history_input': {'ticker': 'TSLA', 'period': '1mo'}}
content='| Date                      |   Open |   High |    Low |   Close |      Volume |   Dividends |   Stock Splits |\n|:--------------------------|-------:|-------:|-------:|--------:|------------:|------------:|---------------:|\n| 2026-01-20 00:00:00-05:00 | 429.36 | 430.73 | 417.44 |  419.25 | 6.31873e+07 |           0 |              0 |\n| 2026-01-21 00:00:00-05:00 | 421.66 | 438.2  | 419.62 |  431.44 | 6.8124e+07  |           0 |              0 |\n| 2026-01-22 00:00:00-05:00 | 435.16 | 449.5  | 432.63 |  449.36 | 7.15467e+07 |           0 |              0 |\n| 2026-01-23 00:00:00-05:00 | 447.43 | 452.43 | 444.04 |  449.06 | 5.67714e+07 |           0 |              0 |\n| 2026-01-26 00:00:00-05:00 | 445    | 445.04 | 434.28 |  435.2  | 4.93974e+07 |           0 |              0 |\n| 2026-01-27 00:00:00-05:00 | 437.41 | 437.52 | 430.69 |  430.9  | 3.77331e+07 |           0 |              0 |\n| 2026-01-28 00:00:00-05:0

In [15]:
llm_with_tools.invoke(messages)

AIMessage(content='테슬라(TSLA)의 주가는 한 달 전에 비해 다음과 같이 변동했습니다:\n\n- **2026년 1월 20일**: 종가 419.25\n- **2026년 2월 17일**: 종가 410.63\n\n주가는 한 달 전에 비해 **내렸습니다.**', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 1517, 'total_tokens': 1592, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'id': 'chatcmpl-DAXj71T5wc9xoWVqiZ4dNEp1D9DQC', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c6ffb-c5e8-7a51-ab64-090a3e66dbaf-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 1517, 'output_tokens': 75, 'total_tokens': 1592, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning

In [16]:
for c in llm.stream([HumanMessage("잘 지냈어? 한국 사회의 문제점이 무엇인지 이야기해줘.")]):
    print(c.content, end="|")

|안|녕하세요|!| 한국| 사회|의| 문제|점|은| 여러| 가지|가| 있지만|,| 몇| 가지| 주요| 문제|를| 살|펴|볼| 수| 있습니다|.

|1|.| **|고|령|화| 사회|**|:| 한국|은| 세계|에서| 가장| 빠|르게| 고|령|화|가| 진행|되고| 있는| 나라| 중| 하나|입니다|.| 이에| 따른| 경제|적| 부담|과| 노|인|복|지| 문제|,| 세|대| 간| 갈|등| 등이| 심|각|합니다|.

|2|.| **|청|년| 실|업|**|:| 젊|은| 세|대|의| 취|업|이| 어려|워|지고| 있으며|,| 이는| 사회|적| 불|만|과| 경제|적| 불|안|정|으로| 이어|집|니다|.| 고|학|력|자| 실|업|률|이| 높은| 것도| 큰| 문제|입니다|.

|3|.| **|주|거| 문제|**|:| 특히| 대|도|시|에서|의| 집|값| 상승|은| 많은| 사람|들에게| 큰| 부담|이| 되고| 있으며|,| 젊|은| 층|의| 주|거| 안정|성을| 위|협|하고| 있습니다|.

|4|.| **|성|차|별|과| 성|폭|력|**|:| 한국| 사회|에서는| 여|전히| 성|별|에| 따른| 차|별|과| 성|폭|력| 문제가| 심|각|하게| 존재|하고| 있습니다|.| 이에| 대한| 사회|적| 인|식|이| 점|차| 변화|하고| 있지만|,| 여|전히| 해결|해야| 할| 문제가| 많|습니다|.

|5|.| **|환경| 문제|**|:| 대|기| 오|염|,| 플|라|스|틱| 폐|기|물| 문제| 등| 환경| 문제|도| 점|점| 더| 심|각|해|지고| 있습니다|.| 지속| 가능한| 발전|을| 위한| 노|력이| 필요|합니다|.

|6|.| **|정|치|적| 갈|등|**|:| 정치|적인| polarization|이| 심|화|되|면서| 사회|적인| 갈|등|과| 대|립|이| 커|지고| 있습니다|.| 이러한| 상황|은| 사회| 통|합|에| 부|정|적인| 영향을| 미|치|고| 있습니다|.

|이|러한| 문제|들은| 복|합|적으로| 얽|혀| 있으며|,| 해결|하기| 위해|서는| 사회| 전|반|의| 노|력이| 

In [17]:
messages = [
    SystemMessage('너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.'),
    HumanMessage("부산은 지금 몇 시야?")
]

response = llm_with_tools.stream(messages)

is_first = True

for chunk in response:
    print("chunk type: ", type(chunk))

    if is_first: 
        is_first = False
        gathered = chunk
    else: 
        gathered += chunk
    
    print('content: ', gathered.content, "tool_call_chunk", gathered.tool_calls)

messages.append(gathered)

chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {}, 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': ''}, 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'type': 'tool_call'}]
chunk type:  <class 'langchain_core.messages.ai.AIMessageChunk'>
content:   tool_call_chunk [{'name': 'get_current_time', 'args': {'timezone': 'Asia'}, 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'type': 'tool_c

In [18]:
gathered

AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'service_tier': 'default'}, id='lc_run--019c6ffc-0863-7920-a233-df826b1fe99a', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 195, 'output_tokens': 23, 'total_tokens': 218, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'index': 0, 'type': 'tool_call_chunk'}], chunk_position='last')

In [19]:
for tool_call in gathered.tool_calls:
    selected_tool = tool_dict[tool_call['name']]

    print(tool_call['args'])

    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

{'timezone': 'Asia/Seoul', 'location': '부산'}
Asia/Seoul (부산) 현재 시각 2026-02-18 18:07:02


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='부산은 지금 몇 시야?', additional_kwargs={}, response_metadata={}),
 AIMessageChunk(content='', additional_kwargs={}, response_metadata={'model_provider': 'openai', 'finish_reason': 'tool_calls', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_373a14eb6f', 'service_tier': 'default'}, id='lc_run--019c6ffc-0863-7920-a233-df826b1fe99a', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Seoul', 'location': '부산'}, 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 195, 'output_tokens': 23, 'total_tokens': 218, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}, tool_call_chunks=[{'name': 'get_current_time', 'args': '{"timezone":"Asia/Seoul","location":"부산"}', 'id': 'call_tQoSB9aOAo9Bmo33kLOBgyOv', 'index': 0, 't

In [20]:
for c in llm_with_tools.stream(messages):
    print(c.content, end="|")

|부|산|은| 지금| |202|6|년| |2|월| |18|일| |18|시| |07|분|입니다|.||||